# Finalise facebook comments
Finalises the facebook comments.
And persists it to s3.
See docs for more information.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime

import pandas as pd
import tentaclio

from phoenix.common import artifacts, run_params
from phoenix.common import utils
from phoenix.tag import export
from phoenix.tag import finalise, finalise_facebook_comments
from phoenix.tag import object_filters

In [ ]:
utils.setup_notebook_output()
utils.setup_notebook_logging()

In [ ]:
# Parameters
# See phoenix/common/run_datetime.py expected format of parameter
RUN_DATETIME = None
TENANT_ID = None

# See phoenix/common/artifacts/registry_environment.py expected format of parameter
ARTIFACTS_ENVIRONMENT_KEY = "local"

# Filters for batch
YEAR_FILTER = 2021
# Without zero padding
MONTH_FILTER = 8
OBJECT_TYPE = "facebook_comments"

# OUTPUT
FINAL_URL_FACEBOOK_COMMENTS = None
INCLUDE_OBJECTS_TENSIONS = True
INCLUDE_SENTIMENT = True
INCLUDE_OBJECTS_TOPICS = True

In [ ]:
cur_run_params = run_params.general.create(ARTIFACTS_ENVIRONMENT_KEY, TENANT_ID, RUN_DATETIME)
    
url_config = {
    "YEAR_FILTER": int(YEAR_FILTER),
    "MONTH_FILTER": int(MONTH_FILTER),
    "OBJECT_TYPE": OBJECT_TYPE
}

TAGGING_RUNS_URL_FACEBOOK_COMMENTS_PULLED = cur_run_params.art_url_reg.get_url("tagging_runs-facebook_comments_pulled", url_config)
TAGGING_RUNS_URL_FACEBOOK_POSTS_TOPICS_FINAL = cur_run_params.art_url_reg.get_url("tagging_runs-facebook_posts_topics_final", url_config)
TAGGING_RUNS_URL_PIPELINE_BASE = cur_run_params.art_url_reg.get_url("tagging_runs-pipeline_base", url_config)
TAGGING_RUNS_URL_OBJECTS_TENSIONS = cur_run_params.art_url_reg.get_url("tagging_runs-objects_tensions", url_config)
TAGGING_RUNS_URL_OBJECTS_TOPICS = cur_run_params.art_url_reg.get_url("tagging_runs-objects_topics", url_config)
TAGGING_RUNS_URL_LANGUAGE_SENTIMENT_OBJECTS = cur_run_params.art_url_reg.get_url("tagging_runs-language_sentiment_objects", url_config)
TAGGING_RUNS_URL_FACEBOOK_COMMENTS_FINAL = cur_run_params.art_url_reg.get_url("tagging_runs-facebook_comments_final", url_config)
if not FINAL_URL_FACEBOOK_COMMENTS:
    FINAL_URL_FACEBOOK_COMMENTS = cur_run_params.art_url_reg.get_url("final-facebook_comments", url_config)

In [ ]:
# Display params.
print(
TAGGING_RUNS_URL_FACEBOOK_COMMENTS_PULLED,
TAGGING_RUNS_URL_FACEBOOK_POSTS_TOPICS_FINAL,
TAGGING_RUNS_URL_PIPELINE_BASE,
TAGGING_RUNS_URL_OBJECTS_TENSIONS,
TAGGING_RUNS_URL_OBJECTS_TOPICS,
TAGGING_RUNS_URL_LANGUAGE_SENTIMENT_OBJECTS,
TAGGING_RUNS_URL_FACEBOOK_COMMENTS_FINAL,
FINAL_URL_FACEBOOK_COMMENTS,
cur_run_params.run_dt.dt,
sep='\n',
)

In [ ]:
facebook_comments_df = artifacts.dataframes.get(TAGGING_RUNS_URL_FACEBOOK_COMMENTS_PULLED).dataframe

In [ ]:
facebook_comments_df.head()

In [ ]:
objects = None
if INCLUDE_OBJECTS_TOPICS and not INCLUDE_OBJECTS_TENSIONS:
    objects = artifacts.dataframes.get(TAGGING_RUNS_URL_OBJECTS_TOPICS).dataframe
    display(objects.head())

In [ ]:
if INCLUDE_OBJECTS_TENSIONS:
    objects = artifacts.dataframes.get(TAGGING_RUNS_URL_OBJECTS_TENSIONS).dataframe
    display(objects.head())

In [ ]:
language_sentiment_objects = None
if INCLUDE_SENTIMENT:
    language_sentiment_objects = artifacts.dataframes.get(TAGGING_RUNS_URL_LANGUAGE_SENTIMENT_OBJECTS).dataframe
    display(language_sentiment_objects.head())

In [ ]:
facebook_comments_intermediate = finalise.join_objects_to_facebook_comments(
    facebook_comments_df=facebook_comments_df,
    objects_df=objects,
    language_sentiment_objects_df=language_sentiment_objects,
)

In [ ]:
facebook_comments_intermediate.shape

In [ ]:
facebook_posts_topics_df = artifacts.dataframes.get(TAGGING_RUNS_URL_FACEBOOK_POSTS_TOPICS_FINAL).dataframe

In [ ]:
facebook_posts_topics_df.shape

In [ ]:
facebook_posts_topics_df

In [ ]:
inherited_columns = finalise_facebook_comments.inherited_columns_for_facebook_comments(
    posts_topics_df=facebook_posts_topics_df,
)
inherited_columns

In [ ]:
facebook_comments_final = finalise_facebook_comments.inherit_from_facebook_posts_topics_df(
    facebook_posts_topics_df, facebook_comments_intermediate, inherited_columns
)

In [ ]:
facebook_comments_final.head()

In [ ]:
facebook_comments_final.shape

In [ ]:
facebook_comments_final.dtypes

In [ ]:
_ = artifacts.dataframes.persist(FINAL_URL_FACEBOOK_COMMENTS, facebook_comments_final)

In [ ]:
_ = artifacts.dataframes.persist(TAGGING_RUNS_URL_FACEBOOK_COMMENTS_FINAL, facebook_comments_final)

In [ ]:
artifacts.dataframes.read_schema(TAGGING_RUNS_URL_FACEBOOK_COMMENTS_FINAL)